Fake news detector

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import re
from nltk.corpus import stopwords  # the for of in with
from nltk.stem.porter import PorterStemmer #baked baking==bake
from sklearn.feature_extraction.text import TfidfVectorizer #baked=[0,0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files
import pandas as pd
import io

# Upload the file
uploaded = files.upload()

# Dynamically get the file name
filename = list(uploaded.keys())[0]

# Load the CSV file using the actual file name
news_df = pd.read_csv(io.BytesIO(uploaded[filename]))

# Display the first few rows of the dataframe
print(news_df.head())



Saving train.csv to train.csv
   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [ ]:
news_df.shape

(20800, 5)

In [ ]:
news_df.isna().sum()


,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
news_df=news_df.fillna(' ')

In [ ]:
news_df['content']=news_df["author"] + " " +news_df['title']

In [ ]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [ ]:
news_df['content'][20796]

'Benjamin Hoffman N.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times'

In [ ]:
#stemming
import nltk
nltk.download('stopwords')

ps=PorterStemmer()

def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]'," ",content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=" ".join(stemmed_content)
  return stemmed_content


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
news_df['content']=news_df['content'].apply(stemming)


In [ ]:
news_df['content']

,content
0,darrel lucu hous dem aid even see comey letter...
1,daniel j flynn flynn hillari clinton big woman...
2,consortiumnew com truth might get fire
3,jessica purkiss civilian kill singl us airstri...
4,howard portnoy iranian woman jail fiction unpu...
...,...
20795,jerom hudson rapper trump poster child white s...
20796,benjamin hoffman n f l playoff schedul matchup...
20797,michael j de la merc rachel abram maci said re...
20798,alex ansari nato russia hold parallel exercis ...


In [ ]:
X=news_df['content'].values
Y=news_df['label'].values


In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
vector=TfidfVectorizer()
vector.fit(X)
X=vector.transform(X)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=1)

In [ ]:
X_train.shape

(16640, 17128)

In [ ]:
X_test.shape

(4160, 17128)

In [ ]:
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
train_y_pred=model.predict(X_train)
print("train accuracy:",accuracy_score(train_y_pred,y_train))

train accuracy: 0.9868389423076923


In [ ]:
test_y_pred=model.predict(X_test)
print("test accuracy",accuracy_score(test_y_pred,y_test))

test accuracy 0.9764423076923077


In [ ]:
input_data=X_test[10]
prediction=model.predict(input_data)
if prediction[0] == 1:
  print("Fake news")
else:
  print("Real news")

Real news


In [ ]:
news_df['content'][10]

'aaron klein obama organ action partner soro link indivis disrupt trump agenda'

creating a training loop and training a model on batches of data

1.Loop through epochs
2. Loop through training batches,perform training steps,calculate the train loss per batch.
3. Loop through testing batches,perform testing steps,calculate the test loss per batch

4. print out what's happening
5. time it all (for fun)


In [ ]:
#import tqdm for progress bar
from tqdm.auto import tqdm

#set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_cpu=timer()

#set the number of epochs
epochs=3

#create training and test loop

for epoch in tqdm(range(epochs)):
  print(f"Epoch:{epoch}\n------")
  #training
  train_loss=0
  #add a loop


ModuleNotFoundError: No module named 'streamlit'